In [ ]:
from IPython.display import display
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as stats
import seaborn as sns

In [ ]:
# Načtení dat
df = pd.read_pickle("accidents.pkl.gz")
# df

---

# Hypotéza 1:

Na silnicích první třídy se při nehodách umíralo se stejnou pravděpodobností jako dálnicích.

K ověření hypotézy využijte 𝜒<sup>2</sup> test s tím, že také určíte, jestli nehody na silnicích 1. třídy
vedly častěji či méně často k fatální nehodě. V tom vám může pomoct “expected” výstup 𝜒<sup>2</sup>
testu.

Nerozlišujte počet následků, ale pouze nehody, kde došlo k jednomu a více úmrtí
(p13a > 0).


---

Vytvoříme kopii dataframe pro testování této hypotézy se sloupci s nimiž pracujeme dále. 

In [ ]:
# Výběr p36 (druhů silnic) + p13a (úmrtí) do nového dataframe
dfh1 = df[["p13a", "p36"]].copy()

Přidáme boolean sloupce **death** (došlo alespoň k jednomu úmrtí?), **1stclass** (jedná se o silnici 1. třídy?) a **highway** (jedná se o dálnici?).

In [ ]:
# Označení, zda u nehody došlo k jednomu nebo více úmrtí
dfh1["death"] = dfh1["p13a"] > 0

# Označení silnic 1. třídy a dálnic pro výpočet pravděpodobností
dfh1["1stclass"] = dfh1["p36"] == 1
dfh1["highway"] = dfh1["p36"] == 0

Nové sloupce použijeme pro 𝜒<sup>2</sup> test, který uděláme jednotlivě pro dálnice a silnice 1. třídy.

1. Vytvoříme kontigenční tabulky pro vztah mezi typem silnice a úmrtí na ní.
1. Vypočteme ``chi2_contingency`` a uložíme si výsledné "expected" tabulky.

In [ ]:
# Výpočet kontigenčních tabulek
ct_1stclass = pd.crosstab(dfh1["death"], dfh1["1stclass"])
ct_highway = pd.crosstab(dfh1["death"], dfh1["highway"])

# Výpočty pravděpodobností
expected_highway = stats.chi2_contingency(ct_highway)[3]
expected_1stclass = stats.chi2_contingency(ct_1stclass)[3]

print(f"Silnice 1. třídy:\n\tObserved:\n{ct_1stclass}\n\n\tExpected:\n{expected_1stclass}\n\n")
print(f"Dálnice:\n\tObserved:\n{ct_highway}\n\n\tExpected:\n{expected_highway}")

Na základě expected výsledků vypočítáme pravděpodobnosti.

Počítáme se sloupcem, kde daný typ silnice má hodnotu ``True`` a výsledná pravděpodobnost je tedy **``počet_úmrtí / celkový_počet_nehod``** na daném typu silnice (dálnice nebo silnice 1. třídy).

In [ ]:
death_probability_1stclass = expected_1stclass[1, 1] / (expected_1stclass[1, 1] + expected_1stclass[0, 1])

death_probability_highway = expected_highway[1, 1] / (expected_highway[1, 1] + expected_highway[0, 1])

print(f"Pravděpodobnost úmrtí na silnici 1. třídy: {death_probability_1stclass}")
print(f"Pravděpodobnost úmrtí na dálnici: {death_probability_highway}")

Výsledek na základě "expected" výsledků nám ukazuje, že pravděpodobnost úmrtí na dálnici je stejná jako pravděpodobnost úmrtí na silnici 1. třídy.

---

# Hypotéza 2

Při nehodách vozidel značky Škoda je škoda na vozidle nižší než při nehodách vozidel Audi.

Využijte vhodný test a určete, zda je škoda na vozidlech vyšší či nižší a zda je tato vlastnost
na požadované hladině významnosti.


---

Nejdříve si oddělíme do sérií škodu na vozidle (sloupec *p53*) pro dané výrobce (**Škoda** => *39*, **Audi** => *2*).

In [ ]:
dfh2 = df[["p53", "p45a"]].copy()

# Získání škod (sloupec p53) pro dané výrobce: Škoda (39), Audi (2)
damage_skoda = dfh2[dfh2["p45a"] == 39]["p53"]
damage_audi = dfh2[dfh2["p45a"] == 2]["p53"]

Nezávislým **párovým T-testem** porovnáme série škod na vozidlech.

In [ ]:
t_skoda, p_skoda = stats.ttest_ind(damage_skoda, damage_audi, equal_var=False)

display(f"T-test Škoda vs Audi: {t_skoda}, p-value: {p_skoda}")

Dle získaného výsledku T-testu jsou škody na vozidlech značky **Škoda** statisticky významně *menší než* škody na vozidlech **Audi**.